<a href="https://colab.research.google.com/github/duanchi1230/NLP_Project_AI2_Reasoning_Challenge/blob/arc-chi/project_arc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
%tensorflow_version 1.x

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

In [0]:
import datetime
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization
# import tfhub 
import tensorflow_hub as hub
import json
import tensorflow as tf

In [0]:

def DataProcessor(path):
  example = []
  for line in open(path):
    line = json.loads(line)
    for choice in line["question"]["choices"]:
      if choice["label"] == line["answerKey"]:
        example.append(run_classifier.InputExample(line["id"], line["question"]["stem"], choice["text"], "1"))
      if choice["label"] != line["answerKey"]:
        example.append(run_classifier.InputExample(line["id"], line["question"]["stem"], choice["text"], "0"))
  return example

In [0]:
arc_corpus = open("/content/drive/My Drive/CSE576_NLP/Data/ARC_Corpus.txt","r+")

arc_easy_dev = DataProcessor("/content/drive/My Drive/CSE576_NLP/Data/ARC-Easy/ARC-Easy-Dev.jsonl")
arc_easy_test = DataProcessor("/content/drive/My Drive/CSE576_NLP/Data/ARC-Easy/ARC-Easy-Test.jsonl")
arc_easy_train = DataProcessor("/content/drive/My Drive/CSE576_NLP/Data/ARC-Easy/ARC-Easy-Train.jsonl")

# for train_examples in arc_easy_dev:
#   print(train_examples.guid) 
#   print(train_examples.text_a) 
#   print(train_examples.text_b) 
#   print(train_examples.label)


In [0]:
len(arc_easy_test)

In [0]:
OUTPUT_DIR = '/content/drive/My Drive/CSE576_NLP/project-arc-code'
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
label_list = ["0", "1"]

num_train_steps = int(len(arc_easy_train) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
bert_config = modeling.BertConfig.from_json_file('/content/drive/My Drive/CSE576_NLP/'+BERT_MODEL+'/bert_config.json')
init_checkpoint = '/content/drive/My Drive/CSE576_NLP/'+BERT_MODEL+'/bert_model.ckpt'

run_config = tf.contrib.tpu.RunConfig(
      model_dir='/content/drive/My Drive/CSE576_NLP',
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
)

model_fn = run_classifier.model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=init_checkpoint,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False,
    use_one_hot_embeddings=False)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      eval_on_tpu =False,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=TRAIN_BATCH_SIZE,
      eval_batch_size=EVAL_BATCH_SIZE,
      predict_batch_size=PREDICT_BATCH_SIZE)

In [0]:
# Train the model
def model_train(estimator, data_set, label_list, tokenizer, TRAIN_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      data_set, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(data_set)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [0]:
model_train(estimator, arc_easy_train, label_list, tokenizer, TRAIN_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps)

In [0]:
def model_eval(estimator, data_set, label_list, tokenizer, EVAL_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps):
  # Eval the model.
  eval_features = run_classifier.convert_examples_to_features(
      data_set, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(data_set)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(data_set) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))

  return result
  # output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  # with tf.gfile.GFile(output_eval_file, "w") as writer:
  #   print("***** Eval results *****")
  #   for key in sorted(result.keys()):
  #     print('  {} = {}'.format(key, str(result[key])))
  #     writer.write("%s = %s\n" % (key, str(result[key])))

In [0]:
eval_result = model_eval(estimator, arc_easy_dev, label_list, tokenizer, EVAL_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps)

In [0]:
eval_result